In [ ]:
!nvidia-smi

Fri Apr 11 01:08:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   39C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install transformers
!pip install langchain
!pip install chromadb
!pip install pypdf
!pip install xformers
!pip install InstructorEmbedding
!pip install pdf2image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 6.6 MB/s eta 0:

In [ ]:
!pip install torch

In [ ]:
import torch
print(torch.__version__)


2.6.0+cu124


In [ ]:
!pip install auto_gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


In [ ]:
!pip install langchain_community

import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


## Data

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!mkdir pdfs

In [ ]:
!gdown 1mmA_ReCmsJiujVWLOkcxKRDQhp9z6DVE -O pdfs/Film-Contracts.pdf
!gdown 1ckb5NUrg3d3O35-U1kluS59LGMhMqVkt -O pdfs/The-Business-of-Film_3rd.pdf


Downloading...
From: https://drive.google.com/uc?id=1mmA_ReCmsJiujVWLOkcxKRDQhp9z6DVE
To: /content/pdfs/Film-Contracts.pdf
100% 33.6M/33.6M [00:00<00:00, 40.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ckb5NUrg3d3O35-U1kluS59LGMhMqVkt
To: /content/pdfs/The-Business-of-Film_3rd.pdf
100% 5.35M/5.35M [00:00<00:00, 34.8MB/s]


In [ ]:
loader = PyPDFDirectoryLoader("pdfs")
docs = loader.load()
len(docs)

1043

In [ ]:
for doc in docs:
  print(doc.metadata)

{'producer': 'Adobe Acrobat 9.32 Paper Capture Plug-in', 'creator': 'ScandAll PRO V1.7 Update', 'creationdate': '2010-04-30T19:12:30+04:00', 'moddate': '2010-05-16T11:55:30+04:00', 'source': 'pdfs/Film-Contracts.pdf', 'total_pages': 645, 'page': 0, 'page_label': '1'}
{'producer': 'Adobe Acrobat 9.32 Paper Capture Plug-in', 'creator': 'ScandAll PRO V1.7 Update', 'creationdate': '2010-04-30T19:12:30+04:00', 'moddate': '2010-05-16T11:55:30+04:00', 'source': 'pdfs/Film-Contracts.pdf', 'total_pages': 645, 'page': 1, 'page_label': '2'}
{'producer': 'Adobe Acrobat 9.32 Paper Capture Plug-in', 'creator': 'ScandAll PRO V1.7 Update', 'creationdate': '2010-04-30T19:12:30+04:00', 'moddate': '2010-05-16T11:55:30+04:00', 'source': 'pdfs/Film-Contracts.pdf', 'total_pages': 645, 'page': 2, 'page_label': '3'}
{'producer': 'Adobe Acrobat 9.32 Paper Capture Plug-in', 'creator': 'ScandAll PRO V1.7 Update', 'creationdate': '2010-04-30T19:12:30+04:00', 'moddate': '2010-05-16T11:55:30+04:00', 'source': 'pdfs

In [ ]:
#from huggingface_hub import snapshot_download
#snapshot_download(repo_id="hkunlp/instructor-large")

!pip install langchain-huggingface

from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'device': DEVICE}
encode_kwargs = {'normalize_embeddings': True}

embeddings = HuggingFaceEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs=model_kwargs)


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    chunk_size=1024,
    chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

3380

In [ ]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: user 3min 34s, sys: 1.33 s, total: 3min 35s
Wall time: 3min 35s


## Mixtral_8x7B_GPTQ


In [ ]:
model_name_or_path = "Llamarider222/Mixtral_8x7B_GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    # revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

NameError: name 'AutoTokenizer' is not defined

In [ ]:
!nvidia-smi

Fri Apr 11 01:28:32 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P0             29W /   70W |    5488MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.

If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

Device set to use cuda:0
The model 'MistralGPTQForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoX

In [ ]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

<ipython-input-20-c1cd673e8ff4>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})


In [ ]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
chain_type="stuff",
    #retriever=db.as_retriever(search_kwargs={"k": 2}),
    retriever=db.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

## Chat

In [ ]:
result = qa_chain( 'The process of transforming a film from a script to a commercial moviegoes through five steps:'  )

<ipython-input-24-f3fbe6f9cfe6>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain( 'The process of transforming a film from a script to a commercial moviegoes through five steps:'  )
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


I don't see any specific question being asked in this context. Could you please clarify or provide more information about the question?


In [ ]:
print(result["source_documents"])

[Document(metadata={'author': 'Stephen R. Greenwald', 'creationdate': '2022-09-09T19:14:57+05:30', 'creator': 'Adobe InDesign 14.0 (Windows)', 'keywords': 'business of media; OTT; crowdfunding; film distribution; media business; media distribution; film finance; media entrepreneurship; AR; VR', 'moddate': '2022-09-14T14:37:46+05:30', 'page': 27, 'page_label': '9', 'producer': 'Adobe Acrobat Pro DC (32-bit) 22.1.20117', 'source': 'pdfs/The-Business-of-Film_3rd.pdf', 'subject': 'This updated third edition introduces readers to the business of film at every stage of the filmmaking lifecycle A practical, hands-on guide exploring streaming, development, financing trends, regional/global/online distribution, shifting business models, exhibition, multi-platform delivery, marketing, VR/AR, accounting, and more.', 'title': 'The Business of Film; A Practical Introduction', 'total_pages': 398}, page_content='newly minted producers were first and foremost businessmen, with a focus \non profit, not

In [ ]:
result = qa_chain( 'list the key characteristics of the film industry'  )

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The key characteristics of the film industry are:

1. Film as intellectual property
2. Managing creative talent
3. Film as a global product
4. Long development, production, and marketing timeline
5. Market structure - oligopoly with streamers and studios controlling the industry
6. Cyclical nature
7. Price elasticity


In [ ]:
print(result["source_documents"])

[Document(metadata={'producer': 'Adobe Acrobat Pro DC (32-bit) 22.1.20117', 'total_pages': 398, 'keywords': 'business of media; OTT; crowdfunding; film distribution; media business; media distribution; film finance; media entrepreneurship; AR; VR', 'title': 'The Business of Film; A Practical Introduction', 'creator': 'Adobe InDesign 14.0 (Windows)', 'source': 'pdfs/The-Business-of-Film_3rd.pdf', 'moddate': '2022-09-14T14:37:46+05:30', 'page': 53, 'page_label': '35', 'author': 'Stephen R. Greenwald', 'creationdate': '2022-09-09T19:14:57+05:30', 'subject': 'This updated third edition introduces readers to the business of film at every stage of the filmmaking lifecycle A practical, hands-on guide exploring streaming, development, financing trends, regional/global/online distribution, shifting business models, exhibition, multi-platform delivery, marketing, VR/AR, accounting, and more.'}, page_content='DOI: 10.4324/9781003217480-2\nChapter 2\nA Business Overview of Film\nThe film business 

In [ ]:
result = qa_chain( 'give me a contract template for submission release')



/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Based on the provided context, it appears that the document being referred to as a "submission release" is actually a formal written contract. Here is a possible template for a submission release based on the information provided:

[Your Name]
[Your Address]
[City, State ZIP Code]
[Email Address]
[Phone Number]

[Date]

[Production Company Name]
[Address]
[City, State ZIP Code]

RELEASE AND SUBMISSION AGREEMENT ("Agreement")

This Release and Submission Agreement ("Agreement") is made and entered into as of [Date] ("Effective Date"), by and between [Your Name], ("You") and [Production Company Name], ("Production Company").

1. Materials Submitted. You hereby submit to Production Company the following materials ("Materials"): [list of materials].
2. Disclosure. You declare that all important features of the Materials are summarized on Schedule A annexed to this form, and you have disclosed no other features to Production Company.
3. Coverage and Governance. This release covers and gover

In [ ]:
print(result["source_documents"][0].page_content)

the formal written contract.
 3 Y ou declare that all important features of your Material, and the par-
ticular items being submitted by you (e.g., script, outline, treatment, 
drawings, photographs, taped materials, electronic files, etc.) are sum-
marized on Schedule A annexed to this form, and you have disclosed 
no other features to PRODUCTION COMPANY .
 4 Y ou acknowledge that this release covers and governs any and all of 
the Material, whether first submitted to PRODUCTION COMPANY 
contemporaneously with, or prior to, or following, the execution of 
this release, and applies also to any submission of the Material made 
to PRODUCTION COMPANY by another source, directly or indi -
rectly, by or through you, representatives, and assigns.
 5 Y ou warrant and represent that (i) Y ou are the sole author of 
the Material and all elements thereof, and (ii) no third party has 
any rights, title, or interest therein or thereto. I agree to indemnify


## References
https://github.com/PanQiWei/AutoGPTQ/
